# Car test-time prediction

## Loading MB dataset

In [11]:
import os
print(os.getcwd())
os.getcwd()

/content


'/content'

In [12]:
import pandas as pd
data = pd.read_csv('mercedes_test.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'mercedes_test.csv'

In [ ]:
# Figure out "m" and "features"
data
# m=4209, n=376

In [ ]:
data.head(5)

In [ ]:
data.tail(5)

## Data pre-processing

In [ ]:
# Choose categorical data
data_temp = data.select_dtypes(include=['object'])
data_temp

In [ ]:
# Choose categorical data columns
cf = data.select_dtypes(include=['object']).columns
print(cf)

In [ ]:
# To change it into "categorical" data type
data[cf]=data[cf].astype('category')

In [ ]:
# One hot encoding
data = pd.get_dummies(data)
print(data)

In [ ]:
# Obtain X from data (excluding 'ID' and 'y')
X_df = data.drop(['ID','y'],axis=1)
# Obtain y from data
y_df = data['y']
print(X_df)
print(y_df)

In [ ]:
# Convert y_df into binary labels
import numpy as np
TF_vector= (y_df<np.median(y_df))
y_df=TF_vector.astype(float)
print(TF_vector)
print(y_df)

In [ ]:
# Conver data frame into numpy array
X,y = X_df.values, y_df.values
print(X)
print(X.shape)
print(y)
print(y.shape)

## Split into train and test datasets

In [ ]:
from sklearn.model_selection import train_test_split
#X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,stratify=y)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,shuffle=False)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

## Least Squares: Hyperparameter search via cross validation

In [ ]:
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import RandomizedSearchCV

model_LS = RidgeClassifier()
#grid_LS = dict(alpha = [10,1,1e-1,1e-2,1e-3])
grid_LS = {'alpha':[10,1,1e-1,1e-2,1e-3]}
cv_LS = RandomizedSearchCV(model_LS,grid_LS,n_iter=5,cv=5)
#cv_LS = RandomizedSearchCV(model_LS,grid_LS,n_iter=5,cv=5,random_state=42)
cv_LS.fit(X_train,y_train)

In [ ]:
cv_LS.cv_results_ #logs results

In [ ]:
cv_LS.best_params_

## Store logs into csv file

In [ ]:
# Store logs into csv file
import pandas as pd
df_LS = pd.DataFrame.from_dict(cv_LS.cv_results_,orient='columns')
# Select columns to be stored
columns = ['params','mean_test_score','std_test_score','rank_test_score']
df_LS = df_LS[columns]
df_LS.to_csv("logs_LS.csv")

## Save the best model

In [ ]:
best_model_LS=cv_LS.best_estimator_
from joblib import dump
dump(best_model_LS, 'best_model_LS.joblib')

## Load "best_model_LS.joblib'

In [ ]:
from joblib import load
loaded_model_LS = load('best_model_LS.joblib')
loaded_model_LS.score(X_test,y_test)

FileNotFoundError: [Errno 2] No such file or directory: 'best_model_LS.joblib'